# Fine-Tuning de Modelo LLM para Domínio Médico

Este notebook implementa o pipeline completo de fine-tuning de um modelo LLM para tarefas de question-answering médico baseado em evidências científicas.

## Objetivos:
1. Carregar dataset médico formatado no padrão Alpaca
2. Carregar modelo base pré-quantizado (Unsloth)
3. Configurar LoRA para treinamento eficiente
4. Treinar modelo com dados médicos
5. Testar e salvar modelo treinado

## Requisitos:
- GPU com pelo menos 8GB VRAM (recomendado 16GB+)
- CUDA instalado
- Bibliotecas: unsloth, transformers, datasets, trl

## Ordem de Execução:
Execute as células **sequencialmente** (de cima para baixo).


In [ ]:
# ============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS (OPCIONAL)
# ============================================================================
# Execute esta célula apenas se estiver usando Google Colab ou se as
# bibliotecas não estiverem instaladas no seu ambiente local.
#
# Para ambiente local, instale via pip no terminal:
#   pip install 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'
#   pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
#   pip install transformers datasets

# Descomente as linhas abaixo se precisar instalar:
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
# !pip install transformers datasets

print("✅ Dependências verificadas. Se houver erro, instale as bibliotecas acima.")


In [ ]:
# ============================================================================
# CÉLULA 2: IMPORTAÇÕES E CONFIGURAÇÕES
# ============================================================================
# Esta célula importa todas as bibliotecas necessárias e carrega as
# configurações centralizadas do módulo model_config.py

import sys
from pathlib import Path

# Adiciona diretório raiz ao path para imports
sys.path.append(str(Path().absolute().parent.parent))

# Importa bibliotecas principais
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer

# Importa configurações e utilitários locais
from training.model_config import (
    get_model_config, get_lora_config, get_training_config,
    get_dataset_config, get_inference_config
)
from utils.prompts import get_medical_alpaca_prompt, get_instruction_only

print("✅ Bibliotecas importadas com sucesso!")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


In [ ]:
# ============================================================================
# CÉLULA 3: CONFIGURAÇÕES E CAMINHOS
# ============================================================================
# Define caminhos e carrega configurações centralizadas

# Obtém configurações
model_config = get_model_config()
lora_config = get_lora_config()
training_config = get_training_config()
dataset_config = get_dataset_config()

# Define caminhos (ajuste conforme necessário)
BASE_DIR = Path().absolute().parent.parent
FORMATTED_DATASET_PATH = BASE_DIR / "formatted_medical_dataset.json"
MODEL_OUTPUT_DIR = BASE_DIR / "lora_model_medical"
TRAINING_OUTPUT_DIR = BASE_DIR / "outputs"

print("=" * 80)
print("CONFIGURAÇÕES DE FINE-TUNING")
print("=" * 80)
print(f"Modelo: {model_config['default_model']}")
print(f"Max sequence length: {model_config['max_seq_length']}")
print(f"LoRA rank: {lora_config['r']}")
print(f"Learning rate: {training_config['learning_rate']}")
print(f"Max steps: {training_config['max_steps']}")
print(f"Dataset: {FORMATTED_DATASET_PATH}")
print(f"Output model: {MODEL_OUTPUT_DIR}")
print("=" * 80)


In [ ]:
# ============================================================================
# CÉLULA 4: CARREGAMENTO DO DATASET FORMATADO
# ============================================================================
# Carrega o dataset médico já formatado no padrão Alpaca.
# Este dataset deve ter sido gerado pelo script format_dataset.py

if not FORMATTED_DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset não encontrado: {FORMATTED_DATASET_PATH}\n"
        f"Execute primeiro: python preprocessing/format_dataset.py"
    )

print(f"📦 Carregando dataset de: {FORMATTED_DATASET_PATH}")

# load_dataset do Hugging Face carrega JSON diretamente
dataset = load_dataset("json", data_files=str(FORMATTED_DATASET_PATH), split="train")

print(f"✅ Dataset carregado: {len(dataset)} exemplos")
print(f"   Estrutura: {dataset.features}")
print(f"\nExemplo de entrada:")
print(f"   Instruction: {dataset[0]['instruction'][:100]}...")
print(f"   Input: {dataset[0]['input'][:100]}...")
print(f"   Output: {dataset[0]['output'][:100]}...")


In [ ]:
# ============================================================================
# CÉLULA 5: CARREGAMENTO DO MODELO BASE
# ============================================================================
# Carrega modelo pré-quantizado do Unsloth.
# Unsloth fornece modelos otimizados que reduzem uso de memória em ~75%
# mantendo qualidade próxima ao modelo original.

print("=" * 80)
print("CARREGANDO MODELO BASE")
print("=" * 80)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_config['default_model'],
    max_seq_length=model_config['max_seq_length'],
    dtype=model_config['dtype'],
    load_in_4bit=model_config['load_in_4bit'],
)

print("✅ Modelo carregado!")
print(f"   Parâmetros totais: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
# ============================================================================
# CÉLULA 6: CONFIGURAÇÃO LoRA
# ============================================================================
# LoRA (Low-Rank Adaptation) permite treinar apenas ~1-5% dos parâmetros,
# reduzindo drasticamente memória e tempo de treinamento.

print("=" * 80)
print("CONFIGURANDO LoRA")
print("=" * 80)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_config['r'],
    target_modules=lora_config['target_modules'],
    lora_alpha=lora_config['lora_alpha'],
    lora_dropout=lora_config['lora_dropout'],
    bias=lora_config['bias'],
    use_gradient_checkpointing=lora_config['use_gradient_checkpointing'],
    random_state=lora_config['random_state'],
    use_rslora=lora_config['use_rslora'],
    loftq_config=lora_config['loftq_config'],
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA configurado!")
print(f"   Parâmetros treináveis: {trainable_params:,}")
print(f"   Parâmetros totais: {total_params:,}")
print(f"   Fração treinável: {(trainable_params/total_params)*100:.2f}%")


In [ ]:
# ============================================================================
# CÉLULA 7: DEFINIÇÃO DO PROMPT ALPACA MÉDICO
# ============================================================================
# Define a função que formata exemplos do dataset para o formato Alpaca.
# Esta função será aplicada a cada exemplo durante o treinamento.

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    """
    Formata exemplos para o formato Alpaca médico
    
    Combina instruction, input e output em um único texto formatado
    que o modelo aprenderá a gerar.
    """
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Usa template Alpaca médico
        text = get_medical_alpaca_prompt(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

print("✅ Função de formatação definida!")


In [ ]:
# ============================================================================
# CÉLULA 8: PREPARAÇÃO DO DATASET PARA TREINAMENTO
# ============================================================================
# Aplica formatação de prompts a todos os exemplos do dataset

print("=" * 80)
print("FORMATANDO DATASET")
print("=" * 80)

formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names
)

print(f"✅ Dataset formatado: {len(formatted_dataset)} exemplos")
print(f"   Estrutura: {formatted_dataset.features}")

# Mostra exemplo formatado
print(f"\nExemplo de texto formatado (primeiros 500 caracteres):")
print("-" * 80)
print(formatted_dataset[0]['text'][:500] + "...")
print("-" * 80)


In [ ]:
# ============================================================================
# CÉLULA 9: CONFIGURAÇÃO DO TRAINER
# ============================================================================
# Configura o SFTTrainer (Supervised Fine-Tuning Trainer) que gerencia
# todo o processo de treinamento.

print("=" * 80)
print("CONFIGURANDO TRAINER")
print("=" * 80)

training_args = TrainingArguments(
    per_device_train_batch_size=training_config['per_device_train_batch_size'],
    gradient_accumulation_steps=training_config['gradient_accumulation_steps'],
    warmup_steps=training_config['warmup_steps'],
    max_steps=training_config['max_steps'],
    learning_rate=training_config['learning_rate'],
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=training_config['logging_steps'],
    optim=training_config['optim'],
    weight_decay=training_config['weight_decay'],
    lr_scheduler_type=training_config['lr_scheduler_type'],
    seed=training_config['seed'],
    output_dir=str(TRAINING_OUTPUT_DIR),
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=model_config['max_seq_length'],
    dataset_num_proc=dataset_config['dataset_num_proc'],
    packing=dataset_config['packing'],
    args=training_args,
)

print("✅ Trainer configurado!")
print(f"   Batch efetivo: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Max steps: {training_args.max_steps}")
print(f"   Learning rate: {training_args.learning_rate}")


In [ ]:
# ============================================================================
# CÉLULA 10: TREINAMENTO DO MODELO
# ============================================================================
# Inicia o processo de treinamento. Este processo pode levar vários minutos
# ou horas dependendo do tamanho do dataset e performance da GPU.
#
# Durante o treinamento, você verá logs mostrando:
# - Loss (deve diminuir ao longo do tempo)
# - Learning rate atual
# - Progresso (steps completados)

print("=" * 80)
print("INICIANDO TREINAMENTO")
print("=" * 80)
print("⚠️  Este processo pode levar vários minutos ou horas...")
print("-" * 80)

trainer_stats = trainer.train()

print("\n" + "=" * 80)
print("✅ TREINAMENTO CONCLUÍDO")
print("=" * 80)
print(f"Loss final: {trainer_stats.training_loss:.4f}")
print(f"Steps completados: {trainer_stats.global_step}")


In [ ]:
# ============================================================================
# CÉLULA 11: TESTE DO MODELO TREINADO
# ============================================================================
# Testa o modelo com um exemplo médico para verificar a qualidade
# das respostas geradas.

print("=" * 80)
print("TESTANDO MODELO TREINADO")
print("=" * 80)

# Prepara modelo para inferência
FastLanguageModel.for_inference(model)

# Exemplo de teste
example_instruction = get_instruction_only()
example_input = """Contexto: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant produces perforations in its leaves through PCD.
Pergunta: Do mitochondria play a role in remodelling plant leaves during programmed cell death?"""

# Formata prompt (sem resposta, queremos que o modelo gere)
prompt = get_medical_alpaca_prompt(example_instruction, example_input, "")

# Tokeniza e gera
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
inference_cfg = get_inference_config()

outputs = model.generate(
    **inputs,
    max_new_tokens=inference_cfg['max_new_tokens'],
    use_cache=inference_cfg['use_cache'],
)

generated_text = tokenizer.batch_decode(outputs)[0]

print("Prompt de entrada:")
print("-" * 80)
print(prompt[:300] + "...")
print("-" * 80)
print("\nResposta gerada:")
print("-" * 80)
print(generated_text)
print("-" * 80)


In [ ]:
# ============================================================================
# CÉLULA 12: SALVAMENTO DO MODELO
# ============================================================================
# Salva o modelo treinado (apenas adaptadores LoRA) e o tokenizer.
# O modelo salvo pode ser carregado depois para inferência.

print("=" * 80)
print("SALVANDO MODELO")
print("=" * 80)

MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

model.save_pretrained(str(MODEL_OUTPUT_DIR))
tokenizer.save_pretrained(str(MODEL_OUTPUT_DIR))

print(f"✅ Modelo salvo em: {MODEL_OUTPUT_DIR}")
print("\nPara carregar o modelo depois, use:")
print(f"  model, tokenizer = FastLanguageModel.from_pretrained('{MODEL_OUTPUT_DIR}')")


In [ ]:
# ============================================================================
# CÉLULA 13: CARREGAMENTO E TESTE DO MODELO SALVO
# ============================================================================
# Demonstra como carregar o modelo salvo e fazer inferência.
# Esta célula é útil para testar o modelo após reiniciar o ambiente.

print("=" * 80)
print("CARREGANDO MODELO SALVO")
print("=" * 80)

# Carrega modelo salvo
loaded_model, loaded_tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(MODEL_OUTPUT_DIR),
    max_seq_length=model_config['max_seq_length'],
    dtype=model_config['dtype'],
    load_in_4bit=model_config['load_in_4bit'],
)

FastLanguageModel.for_inference(loaded_model)

print("✅ Modelo carregado com sucesso!")

# Testa com outro exemplo
example_instruction = get_instruction_only()
example_input = """Contexto: Assessment of visual acuity depends on the optotypes used for measurement.
Pergunta: What are the differences between Landolt C and Snellen E acuity in strabismus amblyopia?"""

prompt = get_medical_alpaca_prompt(example_instruction, example_input, "")
inputs = loaded_tokenizer([prompt], return_tensors="pt").to("cuda")

# Usa TextStreamer para visualizar geração em tempo real
text_streamer = TextStreamer(loaded_tokenizer)
_ = loaded_model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=inference_cfg['max_new_tokens']
)
